# Computing and inspheres and circumspheres

Under different conditions, various types of spheres containing or contained within a shape can be useful.
For example, for spheres contained in a shape we may be interested in the largest sphere contained by a shape, or the largest *concentric* sphere contained within the shape.
For a polyhedron, we may instead want to find the sphere that touches all the faces, if it exists.

In [ ]:
import coxeter
import fresnel
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def plot_polyhedron_with_sphere(shape, insphere=True):
    """Image a polyhedron along with a sphere contained within it."""
    device = fresnel.Device()
    scene = fresnel.Scene(device)
    
    transparent_material = fresnel.material.Material(
        color=fresnel.color.linear([1, 1, 1]),
        spec_trans=0.95,
        roughness=0.2,
        primitive_color_mix=0.0
    )
    colored_material = fresnel.material.Material(
        color=fresnel.color.linear([0.9,0.714,0.169]),
        roughness=0.8,
    )

    # First make the shape and set up its properties.
    primitive = fresnel.geometry.ConvexPolyhedron(
        scene,
        fresnel.util.convex_polyhedron_from_vertices(shape.vertices),
        N=1,
        outline_width=0.01,
        material=transparent_material if insphere else colored_material,
    )
    primitive.color_by_face = 0.0

    # Now draw the insphere within the shape.
    sphere = fresnel.geometry.Sphere(
        scene,
        N=1,
        material=colored_material if insphere else transparent_material,
    )

    # Make the sphere a little bit smaller than it really is,
    # otherwise you get artifacts near the intersection of the
    # polyhedron and the insphere.
    sphere.radius[:] = [shape.insphere.radius * 0.99 if insphere else shape.circumsphere.radius * 1.01]

    scene.camera = fresnel.camera.fit(scene, view='front')
    tracer = fresnel.tracer.Path(device=device, w=300, h=300)
    return tracer.sample(scene, samples=24, light_samples=40)

The [Platonic solids](https://en.wikipedia.org/wiki/Platonic_solid) are a canonical set of shapes we can use for our analysis.
Conveniently, they can easily be generated using coxeter.
A good example is the dodecahedron.

In [ ]:
dodecahedron = coxeter.families.PlatonicFamily.get_shape("Dodecahedron")

We can query different types of spheres from this shape now:

In [ ]:
# The sphere tangent to all the faces of the polyhedron.
print(dodecahedron.insphere)

# The largest concentric sphere contained in the shape.
print(dodecahedron.maximal_centered_bounded_sphere)

Let's visualize what these shapes look like:

In [ ]:
plot_polyhedron_with_sphere(dodecahedron)

If we instead want to look at spheres _containing_ a shape, we can get those as well.

In [ ]:
# The sphere tangent to all the faces of the polyhedron.
print(dodecahedron.circumsphere)

# The largest concentric sphere contained in the shape.
print(dodecahedron.minimal_centered_bounding_sphere)

In [ ]:
plot_polyhedron_with_sphere(dodecahedron, False)